# **1. Perkenalan Dataset**


### Predictive Lead Scoring

dataset ini berisi 41.188 baris, 21 kolom

1. **Sumber Dataset**:  
   UCI ML Repository : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing
2. **Nama Dataset**:  
   bank-full.csv
3. **Tujuan**:  
   memprediksi kesuksesan telemarketing
4. **Format Dataset**:  
   CSV
5. **Informasi Dataset**: 
   
A. Data Nasabah (Bank Client Data)
Informasi dasar dan demografis nasabah.

- age: Umur nasabah (Numerik).

- job: Jenis pekerjaan nasabah (Kategorikal: "admin.", "blue-collar", "technician", "unknown", dll).

- marital: Status pernikahan (Kategorikal: "married", "single", "divorced", "unknown").

- education: Tingkat pendidikan (Kategorikal: "basic.4y", "high.school", "university.degree", "unknown", dll).

- default: Apakah nasabah memiliki kredit macet? (Kategorikal: "no", "yes", "unknown").

- housing: Apakah nasabah memiliki pinjaman perumahan? (Kategorikal: "no", "yes", "unknown").

- loan: Apakah nasabah memiliki pinjaman pribadi? (Kategorikal: "no", "yes", "unknown").

B. Data Terkait Kontak Terakhir (Current Campaign)
Informasi tentang kontak yang dilakukan di kampanye yang sedang berjalan ini.

- contact: Tipe komunikasi yang digunakan (Kategorikal: "cellular", "telephone").

- month: Bulan terakhir kali nasabah dikontak (Kategorikal: "jan", "feb", ..., "dec").

- day_of_week: Hari terakhir kali nasabah dikontak (Kategorikal: "mon", "tue", "wed", "thu", "fri").

- duration: Durasi telepon pada kontak terakhir (Numerik, dalam detik).

* Catatan Penting: fitur ini (duration) harus dibuang (drop) karena menyebabkan kebocoran data (data leakage), sebab durasi baru diketahui setelah telepon selesai.

C. Atribut Lainnya (Other Attributes)
Informasi histori kampanye sebelumnya dan konteks ekonomi makro.

- campaign: Jumlah kontak yang dilakukan untuk nasabah ini selama kampanye ini (Numerik, termasuk kontak terakhir).

- pdays: Jumlah hari yang berlalu sejak nasabah terakhir dikontak dari kampanye sebelumnya (Numerik; 999 berarti "belum pernah dikontak sebelumnya").

- previous: Jumlah kontak yang dilakukan untuk nasabah ini sebelum kampanye ini (Numerik).

- poutcome: Hasil dari kampanye pemasaran sebelumnya (Kategorikal: "failure", "nonexistent", "success").

D. Konteks Sosial & Ekonomi (Social & Economic Context)
Indikator ekonomi makro saat kontak terakhir dilakukan.

- emp.var.rate: Tingkat variasi pekerjaan (Indikator ekonomi triwulanan).

- cons.price.idx: Indeks harga konsumen (Indikator ekonomi bulanan).

- cons.conf.idx: Indeks kepercayaan konsumen (Indikator ekonomi bulanan).

- euribor3m: Suku bunga Euribor 3 bulan (Indikator ekonomi harian).

- nr.employed: Jumlah karyawan (Indikator ekonomi triwulanan).

E. Variabel Target (Output Variable)
Hasil akhir yang ingin kita prediksi.

- y: Target kita. Apakah nasabah pada akhirnya berlangganan deposito berjangka? (Binary: "yes" atau "no").



# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import zipfile
import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE  # Import SMOTE

# Set Data Display options
pd.set_option('display.max_columns', None)

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
# Simpel & Bersih: Load langsung dari path yang ditemukan
import os
import pandas as pd

# Path yang berhasil ditemukan (Colab)
file_path = 'bank-additional/bank-additional-full.csv'

# Fallback ke path lokal jika tidak ada
if not os.path.exists(file_path):
    file_path = '../dataset/bank-additional-full.csv'

if os.path.exists(file_path):
    df = pd.read_csv(file_path, sep=';')
    display(df.head())
else:
    print("Dataset tidak ditemukan. Pastikan Anda sudah menjalankan sel download di atas atau path sudah benar.")

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
# Check basic info
print("Info Dataset:")
df.info()

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Visualize Target Distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='y', data=df)
plt.title('Distribution of Target Variable (y)')
plt.show()

# Visualize Numerical Distributions
df.hist(bins=20, figsize=(20, 15))
plt.show()

# === Visualisasi Data Kategorikal (Bar Charts) ===
print("\nVisualisasi Data Kategorikal (Bar Charts):")
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    if col != 'y':
        plt.figure(figsize=(10, 5))
        sns.countplot(y=col, data=df, order=df[col].value_counts().index, palette='viridis')
        plt.title(f'Distribution of {col}')
        plt.show()

# === Visualisasi Korelasi (Heatmap) ===
print("\nVisualisasi Korelasi (Heatmap):")
plt.figure(figsize=(12, 10))
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
correlation_matrix = df[numerical_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [ ]:
from imblearn.over_sampling import SMOTE

# === 1. Feature Engineering ===

# Binning 'age'
age_bins = [0, 29, 39, 49, 59, 100]
age_labels = ['<30', '30-39', '40-49', '50-59', '60+']
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

# Konsolidasi 'education'
df['education'] = df['education'].replace(['basic.4y', 'basic.6y', 'basic.9y'], 'basic')

# Transformasi 'pdays'
df['pernah_kontak'] = df['pdays'].apply(lambda x: 'no' if x == 999 else 'yes')

# Fitur Rasio
total_contacts = df['previous'] + df['campaign']
df['previous_ratio'] = df['previous'] / total_contacts
df['previous_ratio'] = df['previous_ratio'].fillna(0)

# Drop duration (Data Leakage)
if 'duration' in df.columns:
    df = df.drop(columns=['duration'])

# Handle Duplicates
df = df.drop_duplicates()

# === 2. Encoding Target ===
le = LabelEncoder()
df['y'] = le.fit_transform(df['y'])

# === 3. Split Data (Train/Test) FIRST ===
# Penting: Split dulu baru Preprocessing untuk mencegah Data Leakage
X = df.drop(['y', 'age', 'pdays'], axis=1)
y = df['y']

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Raw Train shape: {X_train_raw.shape}")
print(f"Raw Test shape: {X_test_raw.shape}")

# === 4. Define Transformers ===
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# ColumnTransformer: Scale Numerals AND Encode Categoricals
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# === 5. Pipeline & Transformation ===
# Fit only on Train, Transform both
print("Preprocessing data...")
X_train_processed = preprocessor.fit_transform(X_train_raw)
X_test_processed = preprocessor.transform(X_test_raw)

# Get feature names
feature_names = preprocessor.get_feature_names_out()

# Convert back to DataFrame (for SMOTE clarity)
if hasattr(X_train_processed, 'toarray'):
   X_train_processed = X_train_processed.toarray()
   X_test_processed = X_test_processed.toarray()
   
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names)

# === 6. SMOTE (Oversampling) ===
# Terapkan SMOTE hanya pada Data Training!
print("Applying SMOTE to Training Data...")
print(f"Before SMOTE: {pd.Series(y_train).value_counts().to_dict()}")

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_df, y_train)

print(f"After SMOTE: {pd.Series(y_train_resampled).value_counts().to_dict()}")

# === 7. Saving Processed Data ===
if not os.path.exists('output'):
    os.makedirs('output')

# A. Data Latih (SMOTE)
train_export = pd.concat([pd.DataFrame(X_train_resampled, columns=feature_names), 
                          pd.Series(y_train_resampled, name='y')], axis=1)

# B. Data Uji (Original)
test_export = pd.concat([X_test_df, 
                         pd.Series(y_test.values, name='y')], axis=1)

# C. Data Full Processed (Tanpa SMOTE - Gabungan Train+Test)
# Hati-hati: Kita gabungkan X_train_df (bukan resampled) dan X_test_df
full_X = pd.concat([X_train_df, X_test_df], ignore_index=True)
full_y = pd.concat([pd.Series(y_train.values), pd.Series(y_test.values)], ignore_index=True)
full_export = pd.concat([full_X, full_y.rename('y')], axis=1)

train_export.to_csv('output/train.csv', index=False)
test_export.to_csv('output/test.csv', index=False)
full_export.to_csv('output/data_processed.csv', index=False)

print(f"Saved train.csv (SMOTE): {train_export.shape}")
print(f"Saved test.csv: {test_export.shape}")
print(f"Saved data_processed.csv (All): {full_export.shape}")